# Tugas Besar IF3270 Pembelajaran Mesin
## Bagian B: Implementasi Mini-batch Gradient Descent

Anggota kelompok:

1. Christine Hutabarat (13520005)
2. Hana Fathiyah (13520047)
3. Yohana Golkaria Nainggolan (13520053)
4. Alifia Rahmah (13520122)


### Deklarasi Kelas

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

class Case:
    def __init__(self, model, input, weights):
      self.model = Model(model["input_size"], model["layers"])
      self.input = input
      self.weights = weights

        # mengembalikan informasi array bobot neuron
      def get_arr_neurons_weights(weights):
        arr_neurons = []
        for i in weights:
           arr_neurons.append(i[1:])
        return arr_neurons
      
      self.neuron_weights = get_arr_neurons_weights(self.weights)
      
      # mengembalikan informasi array bobot bias
      def get_arr_bias_weights(weights):
        arr_bias = []
        for i in weights:
           arr_bias.append(i[0])
        return arr_bias
      
      self.bias_weights = get_arr_bias_weights(self.weights)

    # menggambar struktur jaringan dengan bobot yang disingkat
    def draw_compact_structure(self):
        # input layer
        print(f'x({self.model.input_size})')
        print(' ↓ ')

        # hidden layer
        for i in range(len(self.model.layers)-1):
            print(f'h{i+1}({self.model.layers[i].number_of_neurons})\t= {self.model.layers[i].activation_function}')
            print(' ↓ ')

        # output layer
        print(f'y({self.model.layers[-1].number_of_neurons})\t= {self.model.layers[-1].activation_function}')

    # menggambar struktur jaringan dengan bobot yang lengkap
    def draw_structure(self):
        G = nx.Graph()

        weight_label = {}

        # input layer
        for i in range(self.model.input_size + 1): # neuron+bias
            G.add_node(f'x{i}', pos=(1, i + 1)) # x1, x2, ...

        # hidden layer
        if len(self.model.layers) > 1:
            for i in range(len(self.model.layers) - 1): # layer+bias
                for j in range(self.model.layers[i].number_of_neurons + 1): # neuron+bias
                    G.add_node(f'h{i+1}{j}', pos=((i + 2) * 2, j + 1)) # hi0, hi1, .., hij

                    if (j > 0): # other than bias
                        if (i == 0):
                            # first hidden layer
                            for k in range(self.model.input_size + 1): # input layer neuron
                                G.add_edge(f'x{k}', f'h{i+1}{j}')
                                weight_label[(f'x{k}', f'h{i+1}{j}')] = self.weights[i][k][j-1]
                        else:
                            # previous hidden layer neurons
                            for k in range(self.model.layers[i-1].number_of_neurons + 1):
                                G.add_edge(f'h{i}{k}', f'h{i+1}{j}')
                                weight_label[(f'h{i}{k}', f'h{i+1}{j}')] = self.weights[i][k][j-1]

            # output layer
            for i in range(self.model.layers[-1].number_of_neurons): # neurons in output layer
                G.add_node(f'o{i+1}', pos=((len(self.model.layers) + 1) * 2, i+2))
                for j in range(self.model.layers[-1].number_of_neurons + 1):
                    G.add_edge(f'h{len(self.model.layers)-1}{j}', f'o{i+1}')
                    weight_label[f'h{len(self.model.layers)-1}{j}', f'o{i+1}'] = self.weights[-1][j][i]

        else: # only input-output layer
            for i in range(self.model.layers[-1].number_of_neurons):
                G.add_node(f'o{i+1}', pos=((len(self.model.layers) + 1) * 2, i+2))
                for j in range(self.model.input_size + 1):
                    G.add_edge(f'x{j}', f'o{i+1}')
                    weight_label[f'x{j}', f'o{i+1}'] = self.weights[-1][j][i]

        pos = nx.get_node_attributes(G, 'pos')
        fig, ax = plt.subplots()
        nx.draw_networkx_nodes(G, pos, ax=ax, node_size=1000)
        nx.draw_networkx_edges(G, pos, ax=ax)
        labels = {n: n for n in G.nodes()}
        nx.draw_networkx_labels(G, pos, labels, font_color='white', ax=ax)
        nx.draw_networkx_edge_labels(G, pos, weight_label, ax=ax)
        ax.set_xticks([])
        ax.set_yticks([])

        self.print_weight_labels(weight_label)

        plt.show()

    # menampilkan bobot setiap neuron
    def print_weight_labels(self, weight_label):
        print("Daftar bobot:")

        # filter input weights
        x = dict(filter(lambda w: 'x' in w[0][0], weight_label.items()))
        for i in sorted(x.items()):
            print(i[0], '→', i[1])

        # the rest
        h = dict(filter(lambda w: 'h' in w[0][0], weight_label.items()))
        for i in sorted(h.items()):
            print(i[0], '→', i[1])

class Model:
    def __init__(self, input_size, layers):
        self.input_size = input_size

        # membuat array dari layer
        def create_array_layer(layers):
            arr_layers = []
            for i in range(len(layers)):
                arr_layers.append(Layers(layers[i]["number_of_neurons"], layers[i]["activation_function"]))
            return arr_layers

        self.layers = create_array_layer(layers)
        self.cnt_layers = len(self.layers)

class Layers:
    def __init__ (self, number_of_neurons, activation_function):
        self.number_of_neurons = number_of_neurons
        self.activation_function = activation_function

    # menampilkan informasi layer
    def __str__ (self):
        return f'number of neurons: {self.number_of_neurons}\nactivation_function: {self.activation_function}\n'

class Expect:
    def __init__ (self, output, max_sse):
        self.output = output
        self.max_sse = max_sse

### Deklarasi Fungsi Aktivasi

In [ ]:
# Helper function
from math import exp

# net -> persamaan linear (ax+b+...)

def linear(net):
    return net

def relu(net):
    return max(0, net)

def sigmoid(net):
    return float(1/(1 + exp(net * -1)))

def softmax(net_i, arr_net):
    net_sum = 0
    for i in arr_net:
        net_sum += exp(i)

    return float(exp(net_i)/net_sum)

In [ ]:
# Helper function turunan

def d_linear(_):
    return 1

def d_relu(net):
    if net > 0:
        return 1
    return 0
    
def d_sigmoid(net):
    return sigmoid(net) * (1 - sigmoid(net))

def d_softmax(net_i, arr_net):
    # TODO
    pass

### Deklarasi Fungsi Pembanding (Menghitung SSE)

In [ ]:
def count_sse(predict_output, expect_output):
    sse = 0.0
    
    # print predict_output
    print("predict = [", end = "")
    for i in range(len(expect_output)):
        print(predict_output[i], end="")
        if(i < len(expect_output) - 1):
            print(",", end = " ")
    print("]")

    # print expect_output
    print(f"expect = {expect_output}")

    for i in range(len(expect_output)):
        sse += (expect_output[i] - predict_output[i]) ** 2
    return sse

### Load dari file teks

In [ ]:
import json

def load_file(filename):
    f = open(filename)

    data = json.load(f)

    # get model
    model = data["case"]["model"]

    # get input data
    arr_input = data["case"]["input"]

    # get weights data
    arr_weight = data["case"]["weights"]

    expect_output = data["expect"]["output"]
    expect_max_sse = data["expect"]["max_sse"]

    # create object case
    case = Case(model, arr_input, arr_weight)
    expect = Expect(expect_output, expect_max_sse)

    f.close()

    return case, expect

## 1. Implementasi backpropagation dengan mini-batch gradient descent

## 2. Implementasi Penyimpanan Model

## 3. Pengujian Model